In [56]:
import pandas as pd
import numpy as np
import re
import json
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import xgboost as xgb

In [57]:
pd.set_option('display.max_colwidth', 50)

In [58]:
data = pd.read_csv('../CSE-291-ML-for-Systems/data/runtime/query-runtime-all.csv')
data.head()

,query,runtime (ms),db,engine
0,SELECT DISTINCT COURSEalias0.ADVISORY_REQUIREM...,8.092165,advising,mysql
1,"SELECT DISTINCT COURSEalias0.DEPARTMENT , COUR...",0.772238,advising,mysql
2,"SELECT DISTINCT COURSEalias0.DEPARTMENT , COUR...",0.530958,advising,mysql
3,SELECT COUNT( * ) > 0 FROM COURSE AS COURSEali...,92.988014,advising,mysql
4,"SELECT DISTINCT COURSEalias0.DEPARTMENT , COUR...",1.093864,advising,mysql


### Adding new features 

In [59]:
# converting all queries to lowercase

data['query'] = data['query'].str.lower()
data.head()

,query,runtime (ms),db,engine
0,select distinct coursealias0.advisory_requirem...,8.092165,advising,mysql
1,"select distinct coursealias0.department , cour...",0.772238,advising,mysql
2,"select distinct coursealias0.department , cour...",0.530958,advising,mysql
3,select count( * ) > 0 from course as courseali...,92.988014,advising,mysql
4,"select distinct coursealias0.department , cour...",1.093864,advising,mysql


In [60]:
# checking presence of any expensive operation in a SQL query

complex_keywords = ["join", "distinct", "group by", "count", "order by"]

def check_query_complexity(query):
    words = query.split()
    return int(any(keyword in words for keyword in complex_keywords))

data["query_complexity"] = data["query"].apply(check_query_complexity)
data.head()

,query,runtime (ms),db,engine,query_complexity
0,select distinct coursealias0.advisory_requirem...,8.092165,advising,mysql,1
1,"select distinct coursealias0.department , cour...",0.772238,advising,mysql,1
2,"select distinct coursealias0.department , cour...",0.530958,advising,mysql,1
3,select count( * ) > 0 from course as courseali...,92.988014,advising,mysql,0
4,"select distinct coursealias0.department , cour...",1.093864,advising,mysql,1


In [61]:
data["query_complexity"].value_counts()

query_complexity
1    1411
0     392
Name: count, dtype: int64

In [62]:
data["engine"].value_counts()

engine
mysql     1799
sqlite       4
Name: count, dtype: int64

In [63]:
data["db"].value_counts()

db
atis           1118
geography       257
advising        204
imdb             95
car_1            42
inn_1            36
student_1        27
formula_1        16
restaurants       8
Name: count, dtype: int64

In [64]:
# We still have 4 queries of sqlite type, to proceed ahead we can drop those and later drop the "engine" column altogether
data = data[data['engine'] != 'sqlite']
del data['engine']

In [65]:
data.head()

,query,runtime (ms),db,query_complexity
0,select distinct coursealias0.advisory_requirem...,8.092165,advising,1
1,"select distinct coursealias0.department , cour...",0.772238,advising,1
2,"select distinct coursealias0.department , cour...",0.530958,advising,1
3,select count( * ) > 0 from course as courseali...,92.988014,advising,0
4,"select distinct coursealias0.department , cour...",1.093864,advising,1


### Finding the threshold for determining slow or fast query

In [66]:
from sklearn.cluster import KMeans
import numpy as np

filtered_df = data[data['runtime (ms)'] <= 7]
filtered_df = np.array(filtered_df['runtime (ms)']).reshape(-1, 1)  # Reshape for KMeans

kmeans = KMeans(n_clusters=2, n_init='auto').fit(filtered_df)
threshold = kmeans.cluster_centers_.mean()
print(threshold)

print(np.mean(filtered_df))
print(np.median(filtered_df))

2.7436408301656625
1.5049305898205652
0.9899139404296876


In [67]:
data['runtime_boolean'] = (data['runtime (ms)'] >= threshold).astype(int)
data.head()

,query,runtime (ms),db,query_complexity,runtime_boolean
0,select distinct coursealias0.advisory_requirem...,8.092165,advising,1,1
1,"select distinct coursealias0.department , cour...",0.772238,advising,1,0
2,"select distinct coursealias0.department , cour...",0.530958,advising,1,0
3,select count( * ) > 0 from course as courseali...,92.988014,advising,0,1
4,"select distinct coursealias0.department , cour...",1.093864,advising,1,0


In [68]:
counts = data['runtime_boolean'].value_counts()

print(f"Frequency of 0s (runtime < {threshold} ms)", counts[0])
print(f"Frequency of 0s (runtime > {threshold} ms)", counts[1])

Frequency of 0s (runtime < 2.7436408301656625 ms) 1333
Frequency of 0s (runtime > 2.7436408301656625 ms) 466


### Getting the required table size for each query

In [72]:
with open("data/table_sizes.json", 'r') as json_file:
    table_names = json.load(json_file)

print("Loaded dictionary:", table_names)

Loaded dictionary: {'advising': {'area': 76, 'comment_instructor': 0, 'course': 11453, 'course_offering': 151754, 'course_prerequisite': 43, 'course_tag_count': 0, 'gsi': 0, 'instructor': 9443, 'offering_instructor': 159612, 'program': 1, 'program_course': 39, 'program_requirement': 4, 'semester': 74, 'student': 11, 'student_record': 86}, 'atis': {'aircraft': 43, 'airline': 46, 'airport': 52, 'airport_service': 64, 'city': 46, 'class_of_service': 19, 'code_description': 7, 'compartment_class': 6, 'date_day': 2557, 'days': 233, 'dual_carrier': 23, 'equipment_sequence': 952, 'fare': 16252, 'fare_basis': 60, 'flight': 23457, 'flight_fare': 67230, 'flight_leg': 37021, 'flight_stop': 13564, 'food_service': 374, 'ground_service': 168, 'month': 12, 'restriction': 14, 'state': 26, 'time_interval': 13, 'time_zone': 4}, 'geography': {'border_info': 218, 'city': 386, 'highlow': 51, 'lake': 32, 'mountain': 50, 'river': 149, 'state': 51}, 'restaurants': {'geographic': 167, 'location': 9539, 'restau

In [75]:
def compute_table_size(row, table_names):
    query = row['query']
    db = row['db']
    words = query.split()
    count = 0
    for word in words:
        if word in table_names[db]:
            count += table_names[db][word]
    return count

data['table_size'] = data.apply(lambda row: compute_table_size(row, table_names), axis=1)

In [80]:
del data['db']

In [81]:
data.head()

,query,runtime (ms),query_complexity,runtime_boolean,table_size
0,select distinct coursealias0.advisory_requirem...,8.092165,1,1,11453
1,"select distinct coursealias0.department , cour...",0.772238,1,0,11531
2,"select distinct coursealias0.department , cour...",0.530958,1,0,11531
3,select count( * ) > 0 from course as courseali...,92.988014,0,1,326457
4,"select distinct coursealias0.department , cour...",1.093864,1,0,23060


In [82]:
def preprocess_query(query):
    query = re.sub(r'[^\w\s]', '', query)  # Remove punctuation
    query = re.sub(r'\s+', ' ', query)     # Remove unnecessary whitespace
    return query.lower()

### BoW (CountVectorizer)

A bag of words is one of the popular word embedding techniques of text where each value in the vector would represent the count of words in a document/sentence. In other words, it extracts features from the text. 

**BoW** represents each document (or in this case, SQL query) as a vector where each element represents the count of a word in the vocabulary. It is simple and efficient, especially for smaller datasets. However, BoW ignores word order and semantic relationships between words.

**CountVectorizer** (provided by the scikit-learn library in Python) is used to transform a given text into a vector on the basis of the frequency (count) of each word that occurs in the entire text. It creates a matrix in which each unique word is represented by a column of the matrix, and each text sample from the document is a row in the matrix. The value of each cell is the count of the word in that particular text sample. 

In [83]:
bow_features = data['query'].apply(preprocess_query)
print(bow_features[0])

select distinct coursealias0advisory_requirement coursealias0enforced_requirement coursealias0name from course as coursealias0 where coursealias0department eecs and coursealias0number 595 


In [84]:
vectorizer = CountVectorizer()
bow_features = vectorizer.fit_transform(bow_features)

In [85]:
# Format is "(i, j) k" which means there is a nonzero entry with value k (occurring k times) at row i and column j in the matrix.
print(bow_features)

  (0, 1262)	1
  (0, 681)	1
  (0, 533)	1
  (0, 538)	1
  (0, 542)	1
  (0, 875)	1
  (0, 507)	1
  (0, 317)	1
  (0, 532)	1
  (0, 1411)	1
  (0, 536)	1
  (0, 693)	1
  (0, 303)	1
  (0, 544)	1
  (0, 183)	1
  (1, 1262)	2
  (1, 681)	1
  (1, 542)	1
  (1, 875)	2
  (1, 507)	1
  (1, 317)	3
  (1, 532)	1
  (1, 1411)	2
  (1, 536)	1
  (1, 303)	2
  :	:
  (1797, 1195)	1
  (1797, 1196)	1
  (1797, 1197)	1
  (1797, 887)	1
  (1797, 1198)	1
  (1797, 888)	1
  (1797, 1433)	1
  (1797, 950)	1
  (1798, 1262)	1
  (1798, 875)	1
  (1798, 317)	2
  (1798, 1411)	1
  (1798, 303)	2
  (1798, 496)	1
  (1798, 307)	1
  (1798, 885)	1
  (1798, 886)	1
  (1798, 1195)	1
  (1798, 1196)	1
  (1798, 889)	1
  (1798, 328)	1
  (1798, 1197)	1
  (1798, 887)	1
  (1798, 1199)	1
  (1798, 629)	1


In [86]:
bow_features.shape

(1799, 1437)

In [89]:
from scipy.sparse import hstack

additional_features = data[['query_complexity', 'table_size']].values
final_features = hstack([bow_features, additional_features])
print(final_features.shape)

(1799, 1439)


In [90]:
X_train, X_test, y_train, y_test = train_test_split(features, data['runtime_boolean'], test_size=0.2, random_state=42)

In [91]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1439, 1437)
(360, 1437)
(1439,)
(360,)


In [92]:
# Logistic regression
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8861111111111111


/Users/shreyanshjoshi/anaconda3/envs/sj_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [93]:
# XGB Classifier
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8916666666666667


### Word2Vec

**Word2Vec** represents words as dense, continuous-valued vectors in a high-dimensional space where the similarity between words is captured based on their context in the corpus. It captures semantic relationships and similarities between words, even for words not present in the training data. However, Word2Vec requires more data to train effectively and may be computationally expensive.

In [95]:
# Preprocess SQL queries and tokenize
w2v_features = data['query'].apply(lambda x: x.lower().split())
print(w2v_features[0])

['select', 'distinct', 'coursealias0.advisory_requirement', ',', 'coursealias0.enforced_requirement', ',', 'coursealias0.name', 'from', 'course', 'as', 'coursealias0', 'where', 'coursealias0.department', '=', '"eecs"', 'and', 'coursealias0.number', '=', '595', ';']


In [96]:
# Filtering out punctuations
translator = str.maketrans('', '', string.punctuation)
w2v_features = [[word for word in query if word.translate(translator) != ''] for query in w2v_features]
print(w2v_features[0])

['select', 'distinct', 'coursealias0.advisory_requirement', 'coursealias0.enforced_requirement', 'coursealias0.name', 'from', 'course', 'as', 'coursealias0', 'where', 'coursealias0.department', '"eecs"', 'and', 'coursealias0.number', '595']


In [97]:
from gensim.models import Word2Vec
word2vec_model = Word2Vec(w2v_features, window=5, min_count=1, workers=4)

In [98]:
# Each vector is 100-dimensional (default value chosen by Word2Vec)
word2vec_model.wv['select'].shape

(100,)

In [99]:
# Code to see if a word exists in the vocabulary of our model
print(word2vec_model.wv.__contains__("coursealias0.advisory_requirement"))
print(word2vec_model.wv.__contains__("coursealias0.advisory_requirement1"))

True
False


In [100]:
# List of words in the vocabulary of our W2V model
print(word2vec_model.wv.key_to_index.keys())

print('where' in word2vec_model.wv.key_to_index.keys())
print('sdf' in word2vec_model.wv.key_to_index.keys())

dict_keys(['and', 'as', 'select', 'from', 'where', 'city', 'airport_service', 'distinct', 'flight', 'flightalias0.flight_id', 'in', 'cityalias0', 'cityalias0.city_name', 'cityalias0.city_code', 'airport_servicealias0.city_code', 'airport_servicealias0.airport_code', 'flightalias0', 'airport_servicealias0', 'flightalias0.to_airport', 'flightalias0.from_airport', 'cityalias1', 'cityalias1.city_code', 'cityalias1.city_name', 'airport_servicealias1.city_code', 'airport_servicealias1.airport_code', 'airport_servicealias1', 'days', '1991', 'flightalias0.departure_time', 'date_day', 'flightalias0.flight_days', 'farealias0.fare_id', 'daysalias0.days_code', 'daysalias0.day_name', 'flight_fare', 'fare', 'daysalias0', 'date_dayalias0.day_name', 'date_dayalias0.day_number', 'date_dayalias0.month_number', 'date_dayalias0.year', 'flightalias0.arrival_time', 'date_dayalias0', 'farealias0', 'flight_farealias0.fare_id', 'flight_farealias0.flight_id', 'flightalias0.airline_code', 'flight_farealias0', 'f

In [101]:
word_embeddings = {word: word2vec_model.wv[word] for word in word2vec_model.wv.key_to_index.keys()}
word_embeddings

{'and': array([-0.10619007,  0.53171045, -0.33233833, -0.18963136, -0.53196776,
        -0.10498665, -0.23687777,  0.5824856 ,  0.15115713, -0.06569788,
         0.19126858, -0.23387803, -0.46794984, -0.6198566 , -0.67776895,
        -0.26637232, -0.49870032, -0.44944772,  1.0259501 ,  0.11896957,
        -0.419755  ,  1.7318435 , -1.0164435 ,  0.97242206,  0.6229384 ,
         0.05424883, -0.3804839 , -0.27247086, -0.33063865, -0.9500133 ,
        -0.61434954, -0.1690798 ,  1.1728605 , -1.4192911 , -0.49021885,
        -0.70532024,  0.08170394, -0.4920755 , -0.73778987, -0.96548945,
        -0.26909226,  0.1190968 , -0.8726714 , -0.8357815 ,  0.47356027,
         0.23463205, -0.16332848,  1.062983  ,  0.64123595,  0.892802  ,
        -1.2813362 ,  0.6205585 , -0.46513885,  0.38069227,  0.70333457,
         0.88033336, -0.7384804 , -0.15693216,  0.37328917,  1.9080179 ,
         0.90686744,  0.5236852 , -0.55933464, -0.05033451, -0.55046535,
         0.33956093,  0.64563847,  0.463712 

In [102]:
# number of unique keys (words in vocabulary)
len(word_embeddings.keys())

1492

In [103]:
features = np.array([np.mean([word_embeddings[word] for word in query], axis=0) for query in w2v_features])
print(features.shape)

# For each query (datapoint), we represent it in a 100-dimensional vector

(1799, 100)


In [105]:
additional_features = data[['query_complexity', 'table_size']].values
final_features = np.hstack([features, additional_features])
print(final_features.shape)

(1799, 102)


In [106]:
X_train, X_test, y_train, y_test = train_test_split(features, data['runtime_boolean'], test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1439, 100)
(360, 100)
(1439,)
(360,)


In [107]:
model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8166666666666667


In [108]:
model = xgb.XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8805555555555555
